In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:


import flwr
import torch

from fl_g13.fl_pytorch.editing.centralized_mask import load_mask
from fl_g13.fl_pytorch.editing.centralized_mask import save_mask

2025-06-17 11:37:33.017 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13


In [4]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# DEVICE = "cpu"
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")
# disable_progress_bar()

Training on cuda
Flower 1.17.0 / PyTorch 2.6.0+cu118


# Login wandb

In [5]:
!pip install wandb


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
## read .env file
import dotenv

dotenv.load_dotenv()


True

In [7]:
import wandb

# login by key in .env file
WANDB_API_KEY = dotenv.dotenv_values()["WANDB_API_KEY"]
wandb.login(key=WANDB_API_KEY)

C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\.venv\lib\site-packages\notebook\utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\ADMIN\_netrc
wandb: Currently logged in as: thanhnv-it23 (stefano-gamba-social-politecnico-di-torino) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## Build module local

Build module local such that ClientApp can use it

In [8]:
!pip install -e ..

Obtaining file:///C:/Users/ADMIN/Desktop/BACKUP/study/Italy/polito/classes/20242/deep%20learning/project/source_code/fl-g13

  Installing build dependencies: started


  Installing build dependencies: finished with status 'done'


  Checking if build backend supports build_editable: started

[notice] A new release of pip is available: 25.0.1 -> 25.1.1

  Checking if build backend supports build_editable: finished with status 'done'

[notice] To update, run: python.exe -m pip install --upgrade pip

  Getting requirements to build editable: started


  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for fl_g13 (pyproject.toml): started
  Building editable for fl_g13 (pyproject.toml): finished with status 'done'


### Download missing module for clients

Dino model,that is serialized and sent to client by server, require some modules that have to download from source code of dino model


In [9]:
import os
import urllib.request


def download_if_not_exists(file_path: str, file_url: str):
    """
    Checks if a file exists at the given path. If it does not, downloads it from the specified URL.

    Parameters:
    - file_path (str): The local path to check and save the file.
    - file_url (str): The URL from which to download the file.
    """
    if not os.path.exists(file_path):
        print(f"'{file_path}' not found. Downloading from {file_url}...")
        try:
            urllib.request.urlretrieve(file_url, file_path)
            print("Download complete.")
        except Exception as e:
            print(f"Failed to download file: {e}")
    else:
        print(f"'{file_path}' already exists.")

In [10]:
download_if_not_exists("vision_transformer.py",
                       "https://raw.githubusercontent.com/facebookresearch/dino/refs/heads/main/vision_transformer.py")
download_if_not_exists("utils.py",
                       "https://raw.githubusercontent.com/facebookresearch/dino/refs/heads/main/utils.py")


'vision_transformer.py' already exists.
'utils.py' already exists.


# FL

## Configs

In [11]:

from pathlib import Path

import flwr
import torch
from torch.optim.lr_scheduler import CosineAnnealingLR

from fl_g13.architectures import BaseDino
from fl_g13.editing import SparseSGDM

In [12]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# DEVICE = "cpu"
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")

Training on cuda
Flower 1.17.0 / PyTorch 2.6.0+cu118


In [13]:
DEBUG = False

In [14]:
# Model config

## Model Hyper-parameters
head_layers = 3
head_hidden_size = 512
dropout_rate = 0.0
unfreeze_blocks = 1

## Training Hyper-parameters
batch_size = 128
lr = 1e-3
momentum = 0.9
weight_decay = 1e-5
T_max = 8
eta_min = 1e-5

# FL config
K = 100
C = 0.1
J = 4
num_rounds = 30
partition_type = 'iid'

## only for partition_type = 'shard'
num_shards_per_partition = 10

## Server App config
save_every = 1
fraction_fit = C  # Sample of available clients for training
fraction_evaluate = 0.1  # Sample 50% of available clients for evaluation
min_fit_clients = 10  # Never sample less than 10 clients for training
min_evaluate_clients = 5  # Never sample less than 5 clients for evaluation
min_available_clients = 10  # Wait until all 10 clients are available
device = 'cuda'
## checkpoints directory
current_path = Path.cwd()
model_save_path = current_path / f"../models/fl_dino_baseline/noniid"
checkpoint_dir = model_save_path.resolve()
os.makedirs(checkpoint_dir, exist_ok=True)

## Wandb config
use_wandb = True
wandb_config = {
    # wandb param
    'name': 'FL_Dino_Baseline_iid',
    'project_name': "FL_test_chart",
    # model config param
    "fraction_fit": fraction_fit,
    "lr": lr,
    "momentum": momentum,
    'partition_type': partition_type,
    'K': K,
    'C': C,
    'J': J,
}

# model editing config
model_editing = True
mask_type = 'global'
sparsity = 0.2
mask = None

## simulation run config
NUM_CLIENTS = 100
MAX_PARALLEL_CLIENTS = 10

if DEBUG:
    use_wandb = False
    num_rounds = 2
    J = 4


## Define model , optimizer and loss function

In [15]:
from fl_g13.modeling import load_or_create

# Model
model, start_epoch = load_or_create(
    path=checkpoint_dir,
    model_class=BaseDino,
    model_config=None,
    optimizer=None,
    scheduler=None,
    device=device,
    verbose=True,
)

model.to(DEVICE)

unfreeze_blocks = 12
model.unfreeze_blocks(unfreeze_blocks)
# optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)

# Create a dummy mask for SparseSGDM
init_mask = [torch.ones_like(p, device=p.device) for p in
             model.parameters()]  # Must be done AFTER the model is moved to the device
# Optimizer, scheduler, and loss function
optimizer = SparseSGDM(
    model.parameters(),
    mask=init_mask,
    lr=lr,
    momentum=0.9,
    weight_decay=1e-5
)
criterion = torch.nn.CrossEntropyLoss()
scheduler = CosineAnnealingLR(
    optimizer=optimizer,
    T_max=T_max,
    eta_min=eta_min
)

🔍 Loading checkpoint from C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\models\fl_dino_baseline\noniid\fl_fl_baseline_BaseDino_epoch_200_noniid_1_4.pth
📦 Model class in checkpoint: BaseDino
🔧 Model configuration: {'variant': 'dino_vits16', 'dropout_rate': 0.0, 'head_hidden_size': 512, 'head_layers': 3, 'num_classes': 100, 'unfreeze_blocks': 0, 'activation_fn': 'GELU', 'pretrained': True}


Using cache found in C:\Users\ADMIN/.cache\torch\hub\facebookresearch_dino_main
Using cache found in C:\Users\ADMIN/.cache\torch\hub\facebookresearch_dino_main


➡️ Moved model to device: cuda
✅ Loaded checkpoint from C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\models\fl_dino_baseline\noniid\fl_fl_baseline_BaseDino_epoch_200_noniid_1_4.pth, resuming at epoch 201


## Calculate the centralized mask

In [16]:
from typing import Dict, Any


def compute_mask_stats(mask_dict: Dict[str, torch.Tensor]) -> Dict[str, Any]:
    """
    Computes various statistics for a mask represented as a dictionary
    mapping parameter names to mask tensors.

    Args:
        mask_dict: A dictionary where keys are parameter names (str)
                   and values are mask tensors (torch.Tensor) containing 0s and 1s.

    Returns:
        A dictionary containing overall and layer-wise mask statistics.
    """
    stats = {}

    # --- Overall Statistics ---
    total_elements = 0
    kept_elements_overall = 0  # Elements with value 1
    masked_elements_overall = 0  # Elements with value 0

    for name, mask_tensor in mask_dict.items():
        num_elements = mask_tensor.numel()
        kept_in_layer = torch.sum(mask_tensor == 1).item()
        masked_in_layer = num_elements - kept_in_layer

        total_elements += num_elements
        kept_elements_overall += kept_in_layer
        masked_elements_overall += masked_in_layer

        # --- Layer-wise Statistics ---
        layer_stats = {
            'num_elements': num_elements,
            'kept_elements': kept_in_layer,
            'masked_elements': masked_in_layer,
            'density': kept_in_layer / num_elements if num_elements > 0 else 0.0,
            'sparsity': masked_in_layer / num_elements if num_elements > 0 else 0.0
        }
        stats[name] = layer_stats

    # --- Add Overall Statistics to the result dictionary ---
    stats['overall'] = {
        'total_elements': total_elements,
        'kept_elements': kept_elements_overall,
        'masked_elements': masked_elements_overall,
        'density': kept_elements_overall / total_elements if total_elements > 0 else 0.0,
        'sparsity': masked_elements_overall / total_elements if total_elements > 0 else 0.0
    }

    return stats


def print_mask_stats(stats: Dict[str, Any], layer=False):
    """
    Prints the mask statistics in a readable format.

    Args:
        stats: The dictionary returned by compute_mask_stats.
    """
    if 'overall' not in stats:
        print("Invalid stats dictionary format.")
        return

    overall_stats = stats['overall']
    print("--- Overall Mask Statistics ---")
    print(f"Total Elements: {overall_stats['total_elements']}")
    print(f"Kept Elements (1s): {overall_stats['kept_elements']}")
    print(f"Masked Elements (0s): {overall_stats['masked_elements']}")
    print(f"Overall Density: {overall_stats['density']:.4f}")
    print(f"Overall Sparsity: {overall_stats['sparsity']:.4f}")
    print("-" * 30)

    if not layer:
        return

    print("--- Layer-wise Mask Statistics ---")
    # Sort layer names for consistent output
    layer_names = sorted([name for name in stats if name != 'overall'])
    for name in layer_names:
        layer_stats = stats[name]
        print(f"Layer: {name}")
        print(f"  Num Elements: {layer_stats['num_elements']}")
        print(f"  Kept Elements: {layer_stats['kept_elements']}")
        print(f"  Masked Elements: {layer_stats['masked_elements']}")
        print(f"  Density: {layer_stats['density']:.4f}")
        print(f"  Sparsity: {layer_stats['sparsity']:.4f}")
        print("-" * 20)

In [17]:
from fl_g13.fl_pytorch.editing.centralized_mask import get_client_masks

## config client data set params
client_partition_type = 'shard'  # 'iid' or 'shard' for non-iid dataset
client_num_partitions = 100  # equal to number of client
client_num_shards_per_partition = 10
client_batch_size = 16
client_train_test_split_ratio = 0.2
client_dataset = "cifar100"
client_seed = 42,
client_return_dataset = False,

## config get mask params
mask_model = model
mask_sparsity = 0.8
mask_type = 'global'
mask_rounds = 1
mask_func = None
mask_store_in_cpu = True

return_scores = True

## aggregate
agg_strategy = 'union'
agg_func = None

# if DEBUG:
#     client_num_partitions = 10
#     client_batch_size = 128
#     client_train_test_split_ratio = 0.9


## Compute all masks and scores of clients

In [19]:
masks, scores, _ = get_client_masks(
    ## config client data set params
    client_partition_type=client_partition_type,  # 'iid' or 'shard' for non-iid dataset
    client_num_partitions=client_num_partitions,  # equal to number of client
    client_num_shards_per_partition=client_num_shards_per_partition,
    client_batch_size=client_batch_size,
    client_dataset=client_dataset,
    client_train_test_split_ratio=client_train_test_split_ratio,
    client_seed=client_seed,
    client_return_dataset=client_return_dataset,

    ## config get mask params
    mask_model=mask_model,
    mask_sparsity=mask_sparsity,
    mask_type=mask_type,
    mask_rounds=mask_rounds,
    mask_func=mask_func,
    mask_store_in_cpu=mask_store_in_cpu,

    ## get fisher score
    return_scores=return_scores

)

Computing simple global mask with target sparsity (0.80).
Round 1/1.
	Current round density 0.20%
	Computing the masked fisher score
Computing Masked Fisher Score
Device:cuda:0


Fisher Score: 100%|██████████| 25/25 [00:03<00:00,  8.20batch/s]


	Updating the mask
Computing simple global mask with target sparsity (0.80).
Round 1/1.
	Current round density 0.20%
	Computing the masked fisher score
Computing Masked Fisher Score
Device:cuda:0


Fisher Score: 100%|██████████| 25/25 [00:02<00:00,  9.84batch/s]


	Updating the mask
Computing simple global mask with target sparsity (0.80).
Round 1/1.
	Current round density 0.20%
	Computing the masked fisher score
Computing Masked Fisher Score
Device:cuda:0


...

	Updating the mask
Computing simple global mask with target sparsity (0.80).
Round 1/1.
	Current round density 0.20%
	Computing the masked fisher score
Computing Masked Fisher Score
Device:cuda:0


Fisher Score: 100%|██████████| 25/25 [00:02<00:00,  9.08batch/s]


	Updating the mask
Computing simple global mask with target sparsity (0.80).
Round 1/1.
	Current round density 0.20%
	Computing the masked fisher score
Computing Masked Fisher Score
Device:cuda:0


Fisher Score: 100%|██████████| 25/25 [00:02<00:00,  9.07batch/s]


	Updating the mask


## Aggregate mask and fisher score by sum

In [22]:
from fl_g13.fl_pytorch.editing.centralized_mask import aggregate_by_sum

sum_mask = aggregate_by_sum(masks)
sum_mask

{'backbone.blocks.0.norm1.weight': tensor([100., 100., 100.,   6., 100.,  72.,   3.,   0.,   0.,   0., 100., 100.,
           0.,   0.,   0., 100.,   0.,   0.,   0.,   5., 100., 100.,  31.,  89.,
          58., 100.,  99., 100.,   0., 100.,   0.,   0.,   0.,   0., 100., 100.,
          15.,   7., 100.,  15.,   3., 100., 100.,   0., 100., 100.,   1., 100.,
           0., 100., 100., 100.,  71.,   4., 100.,   0., 100., 100.,  35., 100.,
           0.,   0.,  92., 100., 100.,   0.,   0.,   5.,   8.,   0., 100.,  16.,
           0., 100.,  98.,  97.,  10., 100.,   0., 100.,  82.,   0.,   0., 100.,
           0.,   0.,  81., 100., 100., 100., 100., 100., 100., 100.,   1.,   0.,
         100.,  61., 100.,  99., 100.,   0., 100.,   0., 100., 100., 100., 100.,
         100., 100.,  78., 100., 100.,   9.,   0., 100., 100., 100., 100., 100.,
           0., 100.,   0.,  95., 100.,  51., 100.,  36.,   0.,   0.,  39., 100.,
         100., 100., 100.,   0., 100.,   3., 100., 100., 100.,   3.,  39., 

In [23]:
sum_score = aggregate_by_sum(scores)
sum_score

{'backbone.blocks.0.norm1.weight': tensor([2.4826e-06, 1.3353e-04, 2.0109e-05, 4.0474e-03, 1.1877e-05, 1.5901e-03,
         3.2786e-03, 1.9947e-02, 1.2606e-02, 1.8577e-02, 3.8127e-04, 8.2596e-06,
         3.7961e-01, 1.0377e-02, 9.5216e-02, 8.7050e-07, 6.3333e-01, 1.1367e-02,
         5.6201e-02, 3.6065e-03, 2.1329e-06, 8.1740e-06, 2.2999e-03, 9.7582e-04,
         1.8166e-03, 4.7241e-05, 5.4028e-04, 3.2147e-05, 8.3537e-03, 6.1109e-06,
         9.3152e-02, 1.0776e-01, 7.3906e-03, 1.8279e-02, 1.7679e-04, 2.3588e-04,
         2.9649e-03, 3.7142e-03, 1.9406e-05, 3.3711e-03, 3.2103e-03, 2.9857e-05,
         8.4480e-05, 9.2701e-03, 2.8168e-04, 2.0527e-04, 5.5493e-03, 1.8272e-05,
         4.2268e+00, 1.1523e-04, 1.0820e-04, 2.7802e-06, 1.5753e-03, 5.1380e-03,
         3.0350e-04, 4.5393e-03, 1.6930e-04, 3.1512e-04, 2.3011e-03, 1.2914e-05,
         2.5357e-02, 5.6804e-02, 1.0611e-03, 8.2647e-05, 6.2788e-05, 8.4914e-03,
         1.1179e-02, 4.4173e-03, 3.2793e-03, 9.4531e-03, 6.4641e-05, 2.9771

## Save/load them to file

In [24]:
save_mask(sum_mask, 'centralized_sum_mask.pth')
save_mask(sum_score, 'centralized_sum_score.pth')

In [25]:
sum_mask = load_mask('centralized_sum_mask.pth')
sum_score = load_mask('centralized_sum_score.pth')

In [26]:
sum_mask

{'backbone.blocks.0.norm1.weight': tensor([100., 100., 100.,   6., 100.,  72.,   3.,   0.,   0.,   0., 100., 100.,
           0.,   0.,   0., 100.,   0.,   0.,   0.,   5., 100., 100.,  31.,  89.,
          58., 100.,  99., 100.,   0., 100.,   0.,   0.,   0.,   0., 100., 100.,
          15.,   7., 100.,  15.,   3., 100., 100.,   0., 100., 100.,   1., 100.,
           0., 100., 100., 100.,  71.,   4., 100.,   0., 100., 100.,  35., 100.,
           0.,   0.,  92., 100., 100.,   0.,   0.,   5.,   8.,   0., 100.,  16.,
           0., 100.,  98.,  97.,  10., 100.,   0., 100.,  82.,   0.,   0., 100.,
           0.,   0.,  81., 100., 100., 100., 100., 100., 100., 100.,   1.,   0.,
         100.,  61., 100.,  99., 100.,   0., 100.,   0., 100., 100., 100., 100.,
         100., 100.,  78., 100., 100.,   9.,   0., 100., 100., 100., 100., 100.,
           0., 100.,   0.,  95., 100.,  51., 100.,  36.,   0.,   0.,  39., 100.,
         100., 100., 100.,   0., 100.,   3., 100., 100., 100.,   3.,  39., 

In [27]:
sum_score

{'backbone.blocks.0.norm1.weight': tensor([2.4826e-06, 1.3353e-04, 2.0109e-05, 4.0474e-03, 1.1877e-05, 1.5901e-03,
         3.2786e-03, 1.9947e-02, 1.2606e-02, 1.8577e-02, 3.8127e-04, 8.2596e-06,
         3.7961e-01, 1.0377e-02, 9.5216e-02, 8.7050e-07, 6.3333e-01, 1.1367e-02,
         5.6201e-02, 3.6065e-03, 2.1329e-06, 8.1740e-06, 2.2999e-03, 9.7582e-04,
         1.8166e-03, 4.7241e-05, 5.4028e-04, 3.2147e-05, 8.3537e-03, 6.1109e-06,
         9.3152e-02, 1.0776e-01, 7.3906e-03, 1.8279e-02, 1.7679e-04, 2.3588e-04,
         2.9649e-03, 3.7142e-03, 1.9406e-05, 3.3711e-03, 3.2103e-03, 2.9857e-05,
         8.4480e-05, 9.2701e-03, 2.8168e-04, 2.0527e-04, 5.5493e-03, 1.8272e-05,
         4.2268e+00, 1.1523e-04, 1.0820e-04, 2.7802e-06, 1.5753e-03, 5.1380e-03,
         3.0350e-04, 4.5393e-03, 1.6930e-04, 3.1512e-04, 2.3011e-03, 1.2914e-05,
         2.5357e-02, 5.6804e-02, 1.0611e-03, 8.2647e-05, 6.2788e-05, 8.4914e-03,
         1.1179e-02, 4.4173e-03, 3.2793e-03, 9.4531e-03, 6.4641e-05, 2.9771

## Compute mask by fisher score server-side

In [28]:
from fl_g13.editing.masking import _create_gradiend_mask

density = 0.2
mask_type = 'local'

server_mask = _create_gradiend_mask(sum_score, density=density, mask_type=mask_type)
server_mask

{'backbone.blocks.0.norm1.weight': tensor([1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
         0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1.,
         0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.